<a href="https://colab.research.google.com/github/mortgad/DLVR/blob/main/DataCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive, clone repo

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/mortgad/DLVR.git
%cd DLVR

Cloning into 'DLVR'...
remote: Enumerating objects: 350, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 350 (delta 123), reused 163 (delta 95), pack-reused 142 (from 1)
Receiving objects: 100% (350/350), 16.91 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/content/DLVR


# FairFace

In [ ]:
from utils.fairface import load_fairface
df_fairface_train, df_fairface_val = load_fairface()

Downloaded file saved as /content/train_labels.csv
Downloaded file saved as /content/val_labels.csv


In [ ]:
from utils.fairface import preprocess_fairface
df_fairface_eda, df_fairface_code = preprocess_fairface(df_fairface_train, df_fairface_val)

  0%|          | 0/86744 [00:00<?, ?it/s]

  0%|          | 0/10954 [00:00<?, ?it/s]

  0%|          | 0/86744 [00:00<?, ?it/s]

  0%|          | 0/10954 [00:00<?, ?it/s]

  0%|          | 0/10954 [00:00<?, ?it/s]

  0%|          | 0/10954 [00:00<?, ?it/s]

New Race Mapping: {0: 'White', 1: 'Black', 2: 'Asian', 3: 'Indian', 4: 'Others'}


In [ ]:
df_fairface_code.head()

,file,age_code,gender_code,race_code
0,/content/extracted/train/1.jpg,6,0,2
1,/content/extracted/train/2.jpg,4,1,3
2,/content/extracted/train/3.jpg,1,1,1
3,/content/extracted/train/4.jpg,3,1,3
4,/content/extracted/train/5.jpg,3,1,3


## Display FairFace

In [ ]:
from IPython.display import Image, display

age_codes = list(set(df_fairface_code['age_code']))

for code in age_codes:
    print(f"\n\n------ {code} ------\n\n\n")
    # Display 10 random images
    df_age = df_fairface_code[df_fairface_code['age_code'] == code]
    df_age = df_age.sample(10)
    for index, row in df_age.iterrows():
        image_path = row['file']
        # Display the image
        display(Image(filename=image_path, width=200))

# UTKFace

In [ ]:
from utils.utkface import load_utkface
load_utkface()

Extracted part3.tar.gz
Extracted part2.tar.gz
Extracted part1.tar.gz


In [ ]:
from utils.utkface import preprocess_utkface
df_utkface, df_utkface_raw = preprocess_utkface()

Deleted: /content/extracted/part3/.DS_Store
Deleted: /content/extracted/part3/55_0_0_20170116232725357jpg
Deleted: /content/extracted/part3/44_1_4_20170116235150272.pg
Deleted: /content/extracted/part1/61_3_20170109150557335.jpg
Deleted: /content/extracted/part1/61_1_20170109142408075.jpg
Deleted: /content/extracted/part2/39_1_20170116174525125.jpg
Deleted: /content/extracted/part2/53__0_20170116184028385.jpg
Deleted: /content/extracted/part3/24_0_1_20170116220224657 .jpg
Processing from: /content/extracted/part1
Processing from: /content/extracted/part2
Processing from: /content/extracted/part3


In [ ]:
df_utkface_raw.head()

,file,age_raw,gender_code,race_code,age,age_code,gender,race
0,/content/extracted/part1/85_1_0_20170110183513...,85,1,0,70+,8,Female,White
1,/content/extracted/part1/52_0_0_20170104212221...,52,0,0,50-59,6,Male,White
2,/content/extracted/part1/38_1_3_20170109140636...,38,1,3,30-39,4,Female,Indian
3,/content/extracted/part1/85_1_0_20170110181732...,85,1,0,70+,8,Female,White
4,/content/extracted/part1/39_1_3_20170104232838...,39,1,3,30-39,4,Female,Indian


## Display UTKFace

In [ ]:
from IPython.display import Image, display

age_codes = list(set(df_utkface_raw['age_code']))

for code in age_codes:
    print(f"\n\n------ {code} ------\n\n\n")
    # Display 10 random images
    df_age = df_utkface_raw[df_utkface_raw['age_code'] == code]
    df_age = df_age.sample(10)
    for index, row in df_age.iterrows():
        image_path = row['file']
        # Display the image
        display(Image(filename=image_path, width=200))

# IMDB

# Filtering demo

Use opencv to filter images based on the size of the bounding box around faces.

## Sample

In [ ]:
import cv2
import os
from tqdm import tqdm

# Sample image
img_path = df_utkface_raw['file'][0]
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Cascade Classifier
haar_cascade_path = '/content/drive/MyDrive/Deep_Learning_Visual_Recognition/Project/haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Initialize face detector
faces_rect = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=9)

# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 2)

In [ ]:
print('Number of detected faces:', len(faces))\

# loop over all the detected faces
for (x,y,w,h) in faces:

   # To draw a rectangle around the detected face
   cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)

# Display an image in a window
from google.colab.patches import cv2_imshow
cv2_imshow(img)

In [ ]:
# Get area (w * h) of the biggest face
max_area = 0
max_face = None

for (x, y, w, h) in faces:
    area = w * h
    if area > max_area:
        max_area = area
        max_face = (x, y, w, h)

# Print info
print('Max area:', max_area)
print('Max face:', max_face)

Max area: 9604
Max face: (101, 74, 98, 98)


## Make filter 1

Based on a minimum area (w * h)

In [ ]:
min_area = 16000

# Initialize face detector
haar_cascade_path = '/content/drive/MyDrive/Deep_Learning_Visual_Recognition/Project/haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Get list of file paths
file_paths = df_utkface_raw['file'].tolist()

# Subset list for testing
file_paths = file_paths[:100]

# Initialize empty list
filtered_file_paths = []

# Loop over all file paths
for file_path in tqdm(file_paths):
    # Read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 2)

    # Get area (w * h) of the biggest face
    max_area = 0
    max_face = None
    for (x, y, w, h) in faces:
        area = w * h
        if area > max_area:
            max_area = area
            max_face = (x, y, w, h)

    # Check if max_area is greater than min_area
    if max_area > min_area:
        filtered_file_paths.append(file_path)

# Print number of filtered images
print('Number of filtered images:', len(filtered_file_paths))

100%|██████████| 100/100 [00:23<00:00,  4.18it/s]

Number of filtered images: 53


In [ ]:
# Display images
for file_path in filtered_file_paths:
    img = cv2.imread(file_path)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2_imshow(img)

## Make Filter 2

Based on relative size of the largest face

In [ ]:
min_area_percentage = 0.30

# Initialize face detector
haar_cascade_path = '/content/drive/MyDrive/Deep_Learning_Visual_Recognition/Project/haarcascade_frontalface_alt.xml'
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Get list of file paths
file_paths = df_utkface_raw['file'].tolist()

# Subset list for testing
file_paths = file_paths[:100]

# Initialize empty list
filtered_file_paths = []

# Loop over all file paths
for file_path in tqdm(file_paths):
    # Read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 2)

    # Get image area
    img_area = img.shape[0] * img.shape[1]

    # Get area (w * h) of the biggest face
    max_area = 0
    max_face = None
    for (x, y, w, h) in faces:
        area = w * h
        if area > max_area:
            max_area = area
            max_face = (x, y, w, h)

    # Check if the max face area percentage is greater than min_area_percentage
    if max_area / img_area > min_area_percentage:
        filtered_file_paths.append(file_path)

# Print number of filtered images
print('Number of filtered images:', len(filtered_file_paths))

# Display
for file_path in filtered_file_paths:
    img = cv2.imread(file_path)
    cv2_imshow(img)

# Filtering UTKFace and IMDB

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

# Clone GitHub (for utils)
!git clone https://github.com/mortgad/DLVR.git
%cd DLVR

# UTKFace
from utils.utkface import load_utkface
load_utkface()
from utils.utkface import preprocess_utkface
df_utkface, df_utkface_raw = preprocess_utkface()

# IMDB
# ... create utils